## Sample YOLO image inference

In [1]:
import torch
import os

device = "cuda:0" if torch.cuda.is_available() else "cpu"
device

'cuda:0'

In [2]:
from transformers import DetrImageProcessor, DetrForObjectDetection
import torch
from PIL import Image
import shutil
from lat_lon_parser import parse

from PIL.ExifTags import TAGS
# import exifread
from exif import Image as exif

from torchvision.utils import draw_bounding_boxes
from torchvision.transforms.functional import pil_to_tensor, to_pil_image



c:\Users\blain\Documents\GitHub\DeerAim\.venv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:

# assign directory
directory = 'images'
detections_directory = 'images/Detections'
detections_original_directory = 'images/Detections/Originals'
no_detections_directory = 'images/No Detections'

large_animal_list = [19, 20, 21, 23, 24, 25]

In [4]:


# Create the models for object detection

processor = DetrImageProcessor.from_pretrained("facebook/detr-resnet-50")
model = DetrForObjectDetection.from_pretrained("facebook/detr-resnet-50")

# Create the directories to save images, if needed

if (not os.path.exists(detections_directory)):
   # Create a new directory because it does not exist
   os.makedirs(detections_directory)
   print("images/Detections is created!")

if (not os.path.exists(detections_original_directory)):
   # Create a new directory because it does not exist
   os.makedirs(detections_original_directory)
   print("images/Detections is created!")

if not os.path.exists(no_detections_directory):
   # Create a new directory because it does not exist
   os.makedirs(no_detections_directory)
   print("images/No Detections is created!")




The `max_size` parameter is deprecated and will be removed in v4.26. Please specify in `size['longest_edge'] instead`.
Some weights of the model checkpoint at facebook/detr-resnet-50 were not used when initializing DetrForObjectDetection: ['model.backbone.conv_encoder.model.layer1.0.downsample.1.num_batches_tracked', 'model.backbone.conv_encoder.model.layer4.0.downsample.1.num_batches_tracked', 'model.backbone.conv_encoder.model.layer3.0.downsample.1.num_batches_tracked', 'model.backbone.conv_encoder.model.layer2.0.downsample.1.num_batches_tracked']
- This IS expected if you are initializing DetrForObjectDetection from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DetrForObjectDetection from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a Ber

In [91]:

 
# iterate over files in that directory
for filename in os.listdir(directory):
    f = os.path.join(directory, filename)
    # checking if it is a file
    if os.path.isfile(f):

        print(f'Searching for deer in image{filename}')
        image = Image.open(f)
        
        #get the image metadata
        imageData = {}
        try:
            exifdata = exif(f)
        except:
            exifdata = None
            print('\t\timage does not contain metadata')
        
        if exifdata is not None:
            imageData['latitude'] = parse(' '.join(map(str, (exifdata['gps_latitude'] + tuple(exifdata['gps_latitude_ref'])))))
            imageData['longitude'] = parse(' '.join(map(str, (exifdata['gps_longitude'] + tuple(exifdata['gps_longitude_ref'])))))
            imageData['timestamp'] = exifdata['datetime_digitized']

            print(f"\t\tlatitude {round(imageData['latitude'],2):10}")
            print(f"\t\tlongitude {round(imageData['longitude'],2):10}")
            print(f"\t\ttimestamp {imageData['timestamp']}")


            
        inputs = processor(images=image, return_tensors="pt")
        outputs = model(**inputs)

        # convert outputs (bounding boxes and class logits) to COCO API
        # let's only keep detections with score > 0.9
        target_sizes = torch.tensor([image.size[::-1]])
        results = processor.post_process_object_detection(outputs, target_sizes=target_sizes, threshold=0.9)[0]

        # check if the image contains any interesting detections
        if bool(set(large_animal_list).intersection(results['labels'].tolist())):
            print(f'\tAnimal Detected in image{filename}')
            for score, label, box in zip(results["scores"], results["labels"], results["boxes"]):
                box = [round(i, 2) for i in box.tolist()]
                
                # print(
                #         f"Detected {model.config.id2label[label.item()]} with confidence "
                #         f"{round(score.item(), 3)} at location {box}"
                # )

            im = to_pil_image(
                draw_bounding_boxes(
                    pil_to_tensor(image),
                    results['boxes'],
                    colors="red",
                    # labels=[model.config.id2label[x] for x in results['labels'].tolist()],
                    labels = ['Animal Detected' for x in results['labels'].tolist()]
                            )
                        )
            
            # display(im)

            print(f'\tSaving image to {detections_directory}/{filename}')
            im.save(f'{detections_directory}/bbox_{filename}')
            # shutil.move(f'{directory}/{filename}',f'{detections_original_directory}/{filename}')
        else:
            print(f'\tNo Animal Detected in image{filename}')
            print(f'\tMoving image to {no_detections_directory}/{filename}')
            # shutil.move(f'{directory}/{filename}',f'{no_detections_directory}/{filename}')

Searching for deer in image20180512_151348.jpg
		latitude      34.24
		longitude     -86.94
		timestamp 2018:05:12 15:13:48
	No Animal Detected in image20180512_151348.jpg
	Moving image to images/No Detections/20180512_151348.jpg
Searching for deer in imagebus.jpg
		latitude      40.41
		longitude      -3.71
		timestamp 2019:03:01 13:06:33
	No Animal Detected in imagebus.jpg
	Moving image to images/No Detections/bus.jpg
Searching for deer in imagePhoto-6.webp
		image does not contain metadata
	Animal Detected in imagePhoto-6.webp
	Saving image to images/Detections/Photo-6.webp


In [73]:
with open('C:/Users/blain/Documents/GitHub/DeerAim/images/20180512_151348.jpg', 'rb') as f:
    imageData = {}
    exifdata = exif(f)
    if exifdata is not None:
        # for tag in exifdata.list_all():
        #     try:
        #         data = exifdata[tag]     
        #     except:
        #         data = ''
        #     # decode bytes
        #     # if isinstance(data, bytes):
        #     #     data = data.decode()

        #     print(f"{tag:25}: {data}")

        imageData['latitude'] = parse(' '.join(map(str, (exifdata['gps_latitude'] + tuple(exifdata['gps_latitude_ref'])))))
        imageData['longitude'] = parse(' '.join(map(str, (exifdata['gps_longitude'] + tuple(exifdata['gps_longitude_ref'])))))
        imageData['timestamp'] = exifdata['datetime_digitized']
